## Downloading Landsat images (mostly adapted from Quinn's 04_demo_download + stuff found online- glob)

In [ ]:
import pystac
from pystac_client import Client
import planetary_computer
import odc.stac
import matplotlib.pyplot as plt
import os
from pathlib import Path
import urllib
import rioxarray as rxr
import rasterio as rio
from glob import glob

In [ ]:
#Set image directory 
imgdir = f'{Path.home()}/project/data'

if not os.path.exists(imgdir):
    os.makedirs(imgdir)

In [ ]:
#Collection and bands (specific bands for L2 surface reflectance/temperature products)
collection = 'landsat-c2-l2'
asset_id_list = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'ST_B10', 'reduced_resolution_browse']
asset_id_list = ['coastal', 'blue', 'green', 'red', 'nir08', 'swir16', 'lwir11', 'rendered_preview']

#Bounding box
bbox = [104.3,8.3,104.6,8.5]

#Date range
dt = '2026-01-01/2026-02-14'

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",modifier=planetary_computer.sign_inplace)

#Set path, row and cloud cover
results = catalog.search(
    collections=[collection],
    bbox=bbox,
    datetime=dt,
    query={
        "eo:cloud_cover": {"lt": 10},
        "landsat:wrs_path": {"eq": "126"}, # Path 126, Row 54 is the tile that contains Mui Ca Mau National Park.
        "landsat:wrs_row": {"eq": "054"},
    },
)

# Check how many items were returned
items = list(results.items())
print(f"Returned {len(items)} Items")

#Select 1 image to make the stacked image
item = items[0]

In [ ]:
#download bands to imgdir

for asset_id in asset_id_list:
        asset = item.assets[asset_id]
        print(asset.href)

        if asset_id == 'rendered_preview':
            out_fn = item.id + "_rendered_preview.png"
        else:
            out_fn = asset.href.split('?')[0].split('/')[-1]
            
        out_fp = os.path.join(imgdir, out_fn)

        # Check to see if file already exists
        if not os.path.exists(out_fp):
            print("Saving:", out_fp)
            # Download the file
            urllib.request.urlretrieve(asset.href, out_fp)

In [ ]:
#inspect data
!ls -lh $imgdir

In [ ]:
#Stack bands into one TIFF for processing 
#Band paths with glob
band_paths = sorted(glob(os.path.join(imgdir,"**.TIF")))
band_paths

# Read the first band to get metadata for the output file
metadata = rio.open(band_paths[0]).meta.copy()

# Update metadata to reflect the number of bands in the stack
metadata.update(count=len(band_paths))

output_path = os.path.join(imgdir, "LCO8_L2SP_126054_20260129_20260205_02_T1_SR_stacked_rasterio.TIF")

# Write the stacked bands to a new file
stacked_file = rio.open(output_path, 'w', **metadata)

for i, path in enumerate(band_paths, 1):
    source = rio.open(path)
    data = source.read(1)
    stacked_file.write(data, i)
    source.close()

stacked_file.close()

print(f"Successfully created stacked TIFF at: {output_path}")

In [ ]:
stacked_fn = os.path.join(output_path)
stacked_da = rxr.open_rasterio(stacked_fn)
stacked_da